# Machine Learning Pipeline

Prof. Daniel de Abreu Pereira Uhr

### Conteúdo


### Referências

* Géron, Aurélien. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems. O'Reilly Media, 2019.
* Athey, S., & Imbens, G. (2017). "The State of Applied Econometrics: Causality and Policy Evaluation." *Journal of Economic Perspectives*.
* Mullainathan, S., & Spiess, J. (2017). "Machine Learning: An Applied Econometric Approach." *Journal of Economic Perspectives*.
* Varian, H. R. (2014). "Big Data: New Tricks for Econometrics." *Journal of Economic Perspectives*.

In [1]:
# Standard imports
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Transformers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer

# Modeling Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer

# Models
from sklearn.linear_model import LinearRegression, Lasso, BayesianRidge
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn.decomposition import PCA

# XGBoost
from xgboost import XGBRegressor

# Light GBM
from lightgbm import LGBMRegressor

In [2]:
import sklearn, warnings

def get_feature_names(column_transformer):
    """Get feature names from all transformers.
    Returns
    -------
    feature_names : list of strings
        Names of the features produced by transform.
   
    https://johaupt.github.io/scikit-learn/tutorial/python/data%20processing/ml%20pipeline/model%20interpretation/columnTransformer_feature_names.html
    """
    # Remove the internal helper function
    #check_is_fitted(column_transformer)
    
    # Turn loopkup into function for better handling with pipeline later
    def get_names(trans):
        # >> Original get_feature_names() method
        if trans == 'drop' or (
                hasattr(column, '__len__') and not len(column)):
            return []
        if trans == 'passthrough':
            if hasattr(column_transformer, '_df_columns'):
                if ((not isinstance(column, slice))
                        and all(isinstance(col, str) for col in column)):
                    return column
                else:
                    return column_transformer._df_columns[column]
            else:
                indices = np.arange(column_transformer._n_features)
                return ['x%d' % i for i in indices[column]]
        if not hasattr(trans, 'get_feature_names_out'):
            # For transformers without a get_features_names method, use the input
            # names to the column transformer
            if column is None:
                return []
            else:
                return [f for f in column]

        return [f for f in trans.get_feature_names_out()]
    
    ### Start of processing
    feature_names = []
    
    # Allow transformers to be pipelines. Pipeline steps are named differently, so preprocessing is needed
    if type(column_transformer) == sklearn.pipeline.Pipeline:
        l_transformers = [(name, trans, None, None) for step, name, trans in column_transformer._iter()]
    else:
        # For column transformers, follow the original method
        l_transformers = list(column_transformer._iter(fitted=True))
    
    
    for name, trans, column, _ in l_transformers: 
        if type(trans) == sklearn.pipeline.Pipeline:
            # Recursive call on pipeline
            _names = get_feature_names(trans)
            # if pipeline has no transformer that returns names
            if len(_names)==0:
                _names = [f for f in column]
            feature_names.extend(_names)
        else:
            feature_names.extend(get_names(trans))
    
    return feature_names

In [3]:
# Set inline graphs
plt.style.use('seaborn')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

C:\Users\danie\AppData\Local\Temp\ipykernel_28140\573609580.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [4]:
# Import data
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")

print(f"Training data: {np.shape(df_train)} \n Testing data: {np.shape(df_test)}")

FileNotFoundError: [Errno 2] No such file or directory: 'data/train.csv'

## Introdução à Inteligência Artificial (AI) e Aprendizado de Máquina (ML)

### Introdução à Inteligência Artificial (Atificial Intelligence - AI)

Inteligência Artificial (AI) é um campo da ciência da computação que se concentra na criação de sistemas que podem executar tarefas que normalmente exigem inteligência humana. Essas tarefas incluem reconhecimento de fala, reconhecimento de imagem, tomada de decisão e tradução de idiomas. A IA é um campo amplo que abrange várias subáreas, como aprendizado de máquina, processamento de linguagem natural, visão computacional e robótica.

Em outras palavras, pode ser definida como um campo amplo da ciência da computação que busca criar sistemas capazes de simular comportamentos humanos inteligentes, como aprendizado, raciocínio, resolução de problemas e tomada de decisões. Com **objetivo** de desenvolver máquinas que podem "pensar" ou resolver problemas de forma independente.

**Escopo** da AI:

* Machine Learning (ML): Aprendizado a partir de dados.
* Planejamento: Tomada de decisões e estratégias.
* Processamento de Linguagem Natural (NLP): Interação entre máquinas e linguagem humana.
* Visão Computacional: Reconhecimento de imagens e vídeos.
* Robótica: Interação física com o mundo.

Vejamos as possíves subáreas da AI:

<div style="text-align:center;">
    <img src="images\IA_ML_DL.png"  alt="Imagem" style="width: 450px;"/>
</div>

Assim, temos que:
* AI: Campo amplo da ciência da computação que busca criar sistemas capazes de simular comportamentos humanos inteligentes.
* ML: Subárea da AI que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados.
* DL: Subárea do ML que se concentra no desenvolvimento de redes neurais profundas para tarefas de aprendizado de máquina.

A economia pode contribuir muito na área de ML. Então, vamos entender melhor o que é ML.

### Introdução ao Aprendizado de Máquina (Machine Learning - ML)

Aprendizado de Máquina (ML) é uma subárea da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados. Em vez de serem explicitamente programados para executar uma tarefa, os modelos de aprendizado de máquina são treinados em um conjunto de dados para aprender padrões e fazer previsões ou tomar decisões.

O **objetivo** do aprendizado de máquina é criar modelos que aprendem automaticamente a partir de dados, melhorando o desempenho com a experiência.

**Escopo** do ML:
* **Supervisionado:** Aprendizado a partir de dados rotulados. Regressão, classificação.
* **Não supervisionado:** Agrupamento, redução de dimensionalidade.
* **Aprendizado por reforço:** Agentes que aprendem por tentativa e erro


### Relação entre AI-ML, ML-DL

A inteligência artificial é um campo amplo que abrange várias subáreas, como aprendizado de máquina, processamento de linguagem natural, visão computacional e robótica. O aprendizado de máquina é uma subárea da inteligência artificial que se concentra no desenvolvimento de algoritmos e modelos que permitem que os computadores aprendam a partir de dados. Ou seja, ML é uma das técnicas utilizadas dentro da IA para alcançar comportamentos inteligentes. 

**Exemplo**:
* AI: Desenvolver um sistema que possa jogar xadrez de maneira estratégica e vencer um grande mestre humano.
* ML: Usar dados de jogos passados para treinar um modelo que preveja os melhores movimentos no xadrez.
* DL: Usar redes neurais profundas para aprender padrões complexos no jogo de xadrez e melhorar a precisão das previsões.

A IA pode usar ML como uma de suas ferramentas, mas também pode adotar outras técnicas não baseadas em aprendizado de dados. 

A relação ML e DL é que o DL é uma subárea do ML que se concentra no desenvolvimento de redes neurais profundas para tarefas de aprendizado de máquina. Redes neurais profundas são modelos de aprendizado de máquina que consistem em várias camadas de neurônios artificiais, permitindo que o modelo aprenda representações complexas dos dados. No DL o pesquisador perderá o controle sobre o modelo, mas ganhará em desempenho.

### Importância do Aprendizado de Máquina na Economia

**Motivação para o uso de ML na Economia**:

* **Análise de Grandes Volumes de Dados (Big Data)**
  * Com a crescente disponibilidade de dados econômicos de alta frequência e alta dimensionalidade (como transações financeiras, dados de redes sociais e registros administrativos), ML oferece ferramentas para analisar essas informações de maneira eficiente.
  * Economistas podem usar ML para identificar padrões complexos que seriam difíceis de detectar com métodos tradicionais.
* **Flexibilidade em Modelos Não Lineares**
  * Muitas relações econômicas não seguem padrões lineares simples. ML permite modelar interações e não linearidades sem precisar especificá-las explicitamente.
  * Exemplos: efeitos de políticas econômicas, preferências de consumidores, dinâmica de preços.
* **Previsão**
  * A previsão de séries temporais econômicas (PIB, inflação, preços de ativos) pode ser aprimorada com técnicas de ML como Redes Neurais e Gradient Boosting.
  * Esses métodos frequentemente superam os modelos econométricos tradicionais, como ARIMA, em termos de precisão preditiva.
* **Personalização e Segmentação**
  * ML permite segmentar populações econômicas para personalização de políticas públicas ou estratégias de mercado.
  * Exemplos: segmentar consumidores para políticas de subsídio, identificar regiões mais vulneráveis economicamente.
* **Estimação Causal**
  * Em áreas como economia do trabalho, ML auxilia na construção de variáveis instrumentais, balanceamento de covariáveis e estimação de efeitos heterogêneos.
  * Métodos como Double Machine Learning (DML) são projetados para lidar com inferência causal em contextos de alta dimensionalidade.

**Possíveis Aplicações do ML na Economia**
* Mercado Financeiro
  * Previsão de preços de ativos, gestão de portfólios e avaliação de risco utilizam extensivamente técnicas de ML.
* Políticas Públicas
  * ML é usado para identificar beneficiários elegíveis para programas sociais, avaliar políticas públicas e detectar fraudes.
* Economia do Trabalho
  * Analisar grandes bases de dados como RAIS e PNAD para prever salários, identificar padrões de emprego ou avaliar impacto de políticas trabalhistas.
* Comércio Eletrônico
  * Previsão de demanda, análise de comportamento do consumidor e precificação dinâmica.
* Modelos de Preferências
  * Análise de preferências reveladas e declaradas para entender escolhas de consumo e comportamento econômico.

**Vantagens do Uso de ML na Economia**
* Melhor desempenho preditivo: Os modelos de ML, quando bem ajustados, frequentemente superam os modelos econométricos tradicionais.
* Automação de tarefas repetitivas: ML pode automatizar tarefas como imputação de dados, detecção de anomalias e análise exploratória.
* Exploração de grandes conjuntos de dados: Ferramentas de ML conseguem trabalhar com alta dimensionalidade sem problemas significativos.
* Capacidade de modelar relações complexas: Não linearidades, interações e efeitos heterogêneos podem ser modelados sem grande esforço.

**Limitações do Uso de ML na Economia**
* Falta de Interpretabilidade: Modelos de ML frequentemente sacrificam interpretabilidade em troca de desempenho preditivo.
  * Muitos modelos de ML, como Redes Neurais, são frequentemente chamados de "caixas-pretas", dificultando a interpretação dos resultados.
* Desafios em Inferência Causal
  * Embora ML seja excelente para previsão, métodos tradicionais de econometria são frequentemente mais adequados para questões de inferência causal.
  * Métodos híbridos, como Double Machine Learning (DML), tentam resolver esse problema.
* Dependência de Grandes Volumes de Dados
* Overfitting
  * Se não gerenciado adequadamente, o treinamento do modelo pode levar a um ajuste excessivo aos dados, comprometendo a generalização.

**Futuro do ML na Economia**

Com o aumento da disponibilidade de dados e o desenvolvimento de métodos híbridos que combinam econometria e ML, espera-se que o uso de ML na economia cresça ainda mais. Técnicas como EconML, DoubleML e métodos de aprendizado profundo em séries temporais estão pavimentando o caminho para novas descobertas e aplicações econômicas.

## Introdução aos Conceitos Básicos de Machine Learning

**Definição**

Em termos gerais, ML pode ser definido como:

> “[Machine Learning é o] ramo de estudos que dá aos computadores a habilidade de aprender sem terem sido explicitamente programados.” (Arthur Samuel, 1959).

ou 

> “É dito que um programa de computador aprende de uma experiência E com respeito a alguma tarefa T e com alguma medida de performance P, se a performance em T, medida por P, melhora com a experiência E.” (Tom Mitchell, 1997).


### Tipos de Problemas em Machine Learning

O aprendizado de máquina pode ser classificado com base na **natureza do problema que os algoritmos tentam resolver**. Esses problemas são organizados em categorias principais: **aprendizado supervisionado, aprendizado não supervisionado, aprendizado semi-supervisionado e aprendizado por reforço**.

#### 1. Aprendizado Supervisionado

No aprendizado supervisionado, o algoritmo aprende a partir de um conjunto de dados de treinamento que contém **pares entrada-saída**, onde as saídas (rótulos - $Y$) estão disponíveis para cada exemplo de entrada.

**Definição Formal:** Seja um conjunto de dados $D = \{(x_i, y_i)\}_{i=1}^n$, onde $x_i \in \mathbb{R}^d$ é o vetor de entrada e $y_i \in \mathbb{R}$ (regressão) ou $y_i \in \{1, 2, \dots, k\}$ (classificação) é a variável de saída. O objetivo é encontrar uma função $f: \mathbb{R}^d \rightarrow \mathbb{R}$ ou $f: \mathbb{R}^d \rightarrow \{1, 2, \dots, k\}$, tal que minimize uma função de perda $\mathcal{L}(y, f(x))$.

A função de perda $\mathcal{L}(y, \hat{y})$ mede a diferença entre a saída predita $\hat{y}=f(x)$ e o rótulo verdadeiro $y$. O objetivo é encontrar os parâmetros do modelo que minimizam a soma ou média das perdas em todo o conjunto de treinamento.

Como exemplo de funções de perda, temos: 
- **Erro Quadrático Médio (Mean Squared Error - MSE):** Penaliza mais os erros grandes, pois eleva o erro ao quadrado.
- **Erro Absoluto Médio (Mean Absolute Error - MAE):** Menos sensível a valores extremos (outliers).
- **Huber Loss:** Combina MSE e MAE para lidar com outliers.
- **Entropia Cruzada (Cross-Entropy):** Usada em problemas de classificação.
- **Hinge Loss:** Utilizada em máquinas de vetores de suporte (SVMs) para classificação binária.

A função de perda é crucial para guiar o processo de otimização e ajustar os parâmetros do modelo. 

**Exemplo prático:**
Problema 1: Você deseja prever o preço de casas com base em características como número de quartos, área construída, localização e idade do imóvel.
* Entradas (Features): Número de quartos, área construída, localização, idade.
* Saída (Target): Preço do imóvel.
* Algoritmos: Regressão Linear, Árvores de Decisão, Redes Neurais.
  * Preparação dos dados (coletar, limpar, transformar variáveis).
  * Treinar o modelo com os dados de treinamento, minimizando a função de perda.
  * Avaliar o modelo com dados de teste (não vistos) para medir a performance.
  * Fazer previsões com novos dados.

Problema 2: Classificar e-mails como "spam" ou "não spam".
* Entradas (Features): Palavras-chave, remetente, assunto, conteúdo.
* Saída (Target): "Spam" (1) ou "Não Spam" (0).
* Algoritmos: Regressão Logística, Naive Bayes, Random Forest.
  * Pré-processar os dados (tokenização, remoção de stopwords, vetorização).
  * Treinar o modelo com os dados de treinamento, minimizando a função de perda.
  * Avaliar o modelo com dados de teste (não vistos) para medir a performance.
  * Fazer previsões com novos e-mails.

As técnicas de aprendizado supervisionado abrangem uma ampla gama de algoritmos, cada um com características e aplicações específicas. Eles podem ser categorizados com base no tipo de tarefa (regressão ou classificação), estrutura do modelo ou abordagem matemática.


| **Categoria**                     | **Algoritmos**                                                                                                                                                     | **Intuição**                                                                                      |
|-----------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| **Regressão**                     | Regressão Linear Simples, Regressão Linear Múltipla, Regressão Logística, Regressão Ridge, Regressão Lasso, Elastic Net, Regressão Polinomial, Regressão Quantílica | Modelar relações entre variáveis explicativas (features) e uma variável dependente (target).    |
| **Árvores de Decisão**            | Decision Tree, Random Forest, Gradient Boosting (GBM), XGBoost, LightGBM, CatBoost, Extra Trees                                                                  | Dividem os dados em subgrupos baseados em condições de decisão, formando estruturas em árvore.   |
| **Máquinas de Vetores**           | Support Vector Machines (SVM) para Classificação, SVM para Regressão (SVR), Nu-SVM, Linear SVM                                                                  | Encontram um hiperplano ótimo que separa classes ou ajusta dados para regressão.                |
| **Redes Neurais**                 | Multi-Layer Perceptron (MLP), Redes Neurais Convolucionais (CNN), Redes Neurais Recorrentes (RNN), Redes LSTM, Redes GRU, Redes Neurais Profundas (DNN)          | Simulam o comportamento de neurônios humanos para identificar padrões complexos em dados.       |
| **Métodos Probabilísticos**       | Naive Bayes (Gaussian, Multinomial, Bernoulli), Hidden Markov Models (HMM), Maximum Entropy Classifier                                                          | Baseiam-se em probabilidade para classificar ou prever eventos.                                 |
| **Métodos Ensemble**              | Bagging, Random Forest, Boosting (Adaboost, Gradient Boosting, XGBoost, LightGBM, CatBoost), Stacking, Blending                                                | Combinam múltiplos modelos para melhorar a precisão e robustez das previsões.                  |
| **Métodos Baseados em Kernel**    | Regressão Kernel, SVM com Kernel (Linear, Polynomial, RBF, Sigmoid), Gaussian Processes                                                                         | Transformam os dados para um espaço dimensional superior para facilitar separações complexas.   |
| **Métodos Bayesianos**            | Bayesian Linear Regression, Bayesian Logistic Regression, Gaussian Processes Regression (GPR)                                                                  | Incorporam inferências probabilísticas para lidar com incertezas nos modelos.                  |
| **Métodos de Series Temporais**   | ARIMA, SARIMA, SARIMAX, Prophet, Regulação Espacial (Spatial Regression)                                                                                       | Lidam com dados sequenciais, considerando dependências temporais ou espaciais.                 |
| **Métodos de Regularização**      | Regressão Lasso, Regressão Ridge, Elastic Net, Regressão Bayesiana Automática                                                                                   | Adicionam penalidades aos modelos para evitar overfitting e melhorar a generalização.          |
| **Métodos de Distância**          | K-Nearest Neighbors (KNN), Classificação de Similaridade Coseno, Análise de Distância Mahalanobis                                                              | Avaliam proximidade ou similaridade entre pontos de dados com base em métricas de distância.    |
| **Álgebras Tensoriais e Matrizes**| Factorização de Matrizes, Análise de Componentes Principais Supervisionada (SPCA)                                                                              | Reduzem dimensionalidade ou capturam relações latentes em dados de alta dimensão.              |
| **Métodos Esparsos**              | Regressão Lasso, Regressão Bayesiana Esparsa                                                                                                                   | Identificam estruturas esparsas em dados, selecionando variáveis relevantes automaticamente.    |



#### 2. Aprendizado Não Supervisionado

O aprendizado não supervisionado é um paradigma de Machine Learning no qual o modelo aprende padrões e estruturas a partir de dados não rotulados. Ao contrário do aprendizado supervisionado, onde as entradas (features) e saídas (rótulos) são fornecidas, no aprendizado não supervisionado, o modelo tenta descobrir relações, grupos ou representações subjacentes nos dados sem ter acesso a rótulos ou categorias pré-definidas.

Objetivos do Aprendizado Não Supervisionado

* Agrupamento (Clustering): Identificar subgrupos ou clusters de dados que compartilham características semelhantes.
  * Exemplo: Dividir clientes em segmentos com base em comportamento de compra.
* Redução de Dimensionalidade:Resumir ou compactar informações, mantendo o máximo possível de variabilidade dos dados.
  * Exemplo: Compressão de dados ou visualização em 2D/3D.
* Anomalias ou Outliers:Detectar padrões que não se ajustam bem ao comportamento esperado.
  * Exemplo: Identificar transações fraudulentas em cartões de crédito.


| **Categoria**              | **Algoritmos**                                                                                     | **Intuição**                                                                                      |
|----------------------------|---------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| **Clustering (Agrupamento)** | K-Means, Hierarchical Clustering, DBSCAN, Mean-Shift, Gaussian Mixture Models (GMM), Spectral Clustering | Identifica grupos ou padrões ocultos em dados não rotulados, agrupando observações semelhantes.  |
| **Redução de Dimensionalidade** | PCA (Principal Component Analysis), t-SNE, UMAP, ICA (Independent Component Analysis), Factor Analysis | Reduz a quantidade de variáveis (features) para simplificar dados, preservando sua estrutura.    |
| **Modelagem de Densidade**   | Gaussian Mixture Models (GMM), Kernel Density Estimation (KDE), Mean-Shift                        | Estima a distribuição de probabilidade subjacente aos dados para identificar padrões.            |
| **Análise de Associação**    | Apriori, FP-Growth                                                                               | Descobre regras e relações frequentes entre variáveis em conjuntos de dados (ex.: cestas de compras). |
| **Modelos Baseados em Grafos**| Spectral Clustering, Community Detection, DeepWalk                                              | Analisam conexões e interações em dados estruturados como grafos, identificando comunidades ou padrões. |
| **Métodos Baseados em Redes Neurais** | Autoencoders, Variational Autoencoders (VAE), Self-Organizing Maps (SOM)                          | Capturam estruturas latentes em dados para compressão, geração ou representação.                |
| **Análise de Séries Temporais** | Dynamic Time Warping (DTW), Hidden Markov Models (HMM), Clusterização de Séries Temporais       | Identifica padrões em dados temporais sem informações rotuladas.                               |
| **Métodos de Anomalias**      | Isolation Forest, One-Class SVM, Robust PCA                                                   | Detectam observações que se desviam significativamente do padrão geral dos dados.               |
| **Modelagem de Tópicos**      | LDA (Latent Dirichlet Allocation), NMF (Non-Negative Matrix Factorization)                     | Descobre tópicos latentes em conjuntos de dados textuais.                                       |



#### 3. Aprendizado Semi-Supervisionado

O aprendizado semi-supervisionado combina características de aprendizado supervisionado e não supervisionado. Ele é especialmente útil quando temos uma grande quantidade de dados não rotulados, mas apenas uma pequena porção rotulada, devido ao custo ou dificuldade de rotular os dados.

Objetivos do Aprendizado Semi-Supervisionado
* Aproveitar os Dados Não Rotulados: Usar a informação contida nos dados não rotulados para complementar os dados rotulados, melhorando a capacidade do modelo.
* Reduzir Custo de Rotulagem: Permitir que modelos sejam treinados de forma eficiente sem a necessidade de rotular todos os dados manualmente.
* Explorar Estruturas Subjacentes: Identificar padrões ou relações nos dados não rotulados para guiar o aprendizado.

Exemplo Prático:
* Problema: Classificar e-mails como "spam" ou "não spam".
* Dados disponíveis: 10% de e-mails rotulados como "spam" ou "não spam"; 90% não rotulados.
* Solução: Um modelo semi-supervisionado usa os 10% rotulados para aprendizado inicial e aproveita padrões nos 90% não rotulados para refinar a classificação.


| **Categoria**             | **Métodos e Algoritmos**                                                                                 | **Intuição**                                                                                       |
|---------------------------|---------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Self-Training**          | Self-Training, Pseudo-Labeling                                                                          | O modelo inicial é treinado com dados rotulados, e as previsões de alta confiança sobre dados não rotulados são usadas para expandir o conjunto rotulado. |
| **Graph-Based Methods**    | Label Propagation, Label Spreading                                                                      | Usa grafos para modelar as relações entre dados rotulados e não rotulados, propagando rótulos através das conexões.                     |
| **Co-Training**            | Co-Training                                                                                            | Treina dois modelos independentes em subconjuntos de features e permite que um modelo rotule os dados não rotulados para o outro.        |
| **Semi-Supervised SVM**    | Semi-Supervised SVM                                                                                    | Expande o SVM tradicional, usando dados não rotulados para encontrar um limite de decisão que maximize a margem.                        |
| **Generative Models**      | Semi-Supervised Variational Autoencoders (VAE), Generative Adversarial Networks (GANs)                 | Modelos generativos que aproveitam a estrutura latente dos dados para aprendizado semi-supervisionado.                                 |



#### 4. Aprendizado por Reforço

O aprendizado por reforço (Reinforcement Learning - RL) é um paradigma no qual um agente aprende a tomar decisões através da interação com um ambiente dinâmico. O objetivo do agente é maximizar uma função de recompensa acumulada ao longo do tempo.

* Elementos do Aprendizado por Reforço
  * **Agente:** Entidade que toma decisões e interage com o ambiente.
  * **Ambiente:** O mundo no qual o agente opera e toma ações.
  * **Estado (State - *s*):** Representação do ambiente em um determinado momento.
  * **Ação (Action - *a*):** Decisão tomada pelo agente em um estado específico.
  * **Recompensa (Reward - *r*):** Feedback imediato que o agente recebe após tomar uma ação.
  * **Política (Policy - *π*):** Estratégia ou comportamento do agente para selecionar ações.
  * **Valor (Value - *V*):** Medida de quão bom é um estado ou ação em um determinado momento.
  * **Função de Q-Valor (Q-Value - *Q*):** Medida de quão bom é escolher uma ação em um determinado estado.

Exemplo Prático
* Problema: Treinar um robô para andar.
* Estados: Posição do robô, inclinação, velocidade.
* Ações: Mover pernas, ajustar direção.
* Recompensa: Distância percorrida com sucesso sem cair.

| **Categoria**             | **Métodos e Algoritmos**                                                                              | **Intuição**                                                                                       |
|---------------------------|------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Baseado em Políticas**   | Policy Gradient, REINFORCE, Proximal Policy Optimization (PPO), Trust Region Policy Optimization (TRPO) | Aprende diretamente a política (\(\pi(a|s)\)) para escolher ações, sem usar a função de valor.    |
| **Baseado em Valores**     | Q-Learning, Deep Q-Networks (DQN), SARSA                                                             | Aprende a função \(Q(s, a)\) para estimar o valor de ações em estados específicos.                |
| **Métodos de Ator-Crítico**| Advantage Actor-Critic (A2C), Asynchronous Actor-Critic (A3C), Soft Actor-Critic (SAC)               | Combina aprendizado de políticas e valores, onde o ator (policy) decide ações, e o crítico avalia.|
| **Exploração Multiarmada** | Bandit Algorithms (Epsilon-Greedy, Upper Confidence Bound - UCB, Thompson Sampling)                  | Resolve problemas onde o agente precisa decidir entre várias opções sem dependência de estados.   |



#### 5. Aprendizado por Transferência (Transfer Learning)

O aprendizado por transferência é uma abordagem onde o conhecimento adquirido em uma tarefa (fonte) é reutilizado para resolver uma nova tarefa (alvo). Essa técnica é especialmente útil quando a tarefa alvo tem poucos dados disponíveis, mas está relacionada a uma tarefa para a qual há um modelo pré-treinado em um grande conjunto de dados.

Aplicações
* Visão Computacional: Usar redes pré-treinadas como ResNet, VGG ou EfficientNet em datasets como ImageNet e adaptá-las para tarefas específicas como classificação médica.
* Processamento de Linguagem Natural: Modelos como BERT, GPT e T5 são treinados em grandes corpora e adaptados para tarefas específicas como análise de sentimentos ou tradução.
* Reconhecimento de Voz: Modelos pré-treinados em grandes bases de dados de fala são adaptados para línguas ou sotaques específicos.


| **Categoria**                   | **Métodos e Algoritmos**                                                                                 | **Intuição**                                                                                       |
|---------------------------------|---------------------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------------|
| **Feature Extraction**          | Transferência de Camadas Congeladas                                                                      | Reutiliza as primeiras camadas de um modelo pré-treinado como extrator de features fixo.         |
| **Fine-Tuning**                 | Fine-Tuning de Redes Neurais, Ajuste de Camadas Específicas                                              | Ajusta as camadas finais de um modelo pré-treinado para uma nova tarefa, mantendo outras fixas.  |
| **Domain Adaptation**           | Adversarial Training for Domain Adaptation (ADDA), Maximum Mean Discrepancy (MMD)                        | Adapta um modelo para funcionar bem em um domínio diferente, reduzindo a discrepância entre domínios. |
| **Zero-Shot Learning**          | CLIP (Contrastive Language–Image Pretraining), GPT                                                      | Usa um modelo pré-treinado em uma tarefa ampla para resolver uma nova tarefa sem treinamento adicional. |
| **Few-Shot Learning**           | Prototypical Networks, Matching Networks, MAML (Model-Agnostic Meta-Learning)                           | Treina modelos para aprender rapidamente com poucos exemplos.                                   |



## Conceitos Fundamentais Relacionados


#### **Validação Cruzada**

**Descrição:**  
A validação cruzada é uma técnica usada para avaliar a performance de um modelo dividindo os dados em partes de treinamento e teste, garantindo que o modelo seja avaliado de forma confiável e generalizável.

**Tipos de Validação Cruzada:**
- **K-Fold Cross-Validation:**  
  Divide os dados em $K$ partes (folds). Em cada iteração, um fold é usado como teste e os outros $K-1$ como treinamento.
  - **Vantagem:** Reduz a variância da avaliação do modelo.
- **Leave-One-Out Cross-Validation (LOOCV):**  
  Cada observação é usada como conjunto de teste uma vez, e todas as outras como treinamento.
  - **Vantagem:** Usa quase todos os dados para treinamento.
  - **Desvantagem:** Computacionalmente caro para grandes datasets.

**Importância:**
- Evita problemas de overfitting e underfitting.
- Proporciona uma estimativa mais confiável da performance do modelo.


#### **Overfitting e Underfitting**

**Overfitting:**  
Ocorre quando o modelo é excessivamente ajustado aos dados de treinamento, capturando ruídos e padrões específicos demais.  
- **Sintomas:** Alta performance no treinamento, mas baixa performance em novos dados.  
- **Exemplo:** Um modelo que ajusta exatamente cada ponto dos dados de treinamento.

**Underfitting:**  
Ocorre quando o modelo não consegue capturar a complexidade dos dados.  
- **Sintomas:** Baixa performance tanto no treinamento quanto no teste.  
- **Exemplo:** Um modelo linear usado para dados com uma relação não-linear.

**Visualização:**
- **Underfitting:** Modelo simples demais (reta em dados não-lineares).
- **Good Fit:** Modelo adequado para os padrões gerais.
- **Overfitting:** Modelo que segue cada ponto dos dados.


#### **Regularização**

**Descrição:**  
Regularização adiciona penalidades aos coeficientes do modelo para evitar que eles assumam valores extremos, reduzindo o risco de overfitting.

**Principais Técnicas:**
- **LASSO (Least Absolute Shrinkage and Selection Operator):**  
  Adiciona uma penalidade $\lambda \sum |\beta_j|$, promovendo coeficientes esparsos (alguns $\beta_j = 0$).
- **Ridge Regression:**  
  Adiciona uma penalidade $\lambda \sum \beta_j^2$, encolhendo os coeficientes, mas não os eliminando.
- **Elastic Net:**  
  Combinação de LASSO e Ridge, com penalidade $\lambda_1 \sum |\beta_j| + \lambda_2 \sum \beta_j^2$.

**Objetivo:**  
Controlar a complexidade do modelo para melhorar sua generalização.


#### **Feature Engineering**

**Descrição:**  
Feature engineering é o processo de criar, selecionar ou transformar variáveis (features) para melhorar a capacidade preditiva de um modelo.

**Passos Importantes:**
1. **Criação de Novas Features:**  
   Combinar variáveis existentes ou criar interações (ex.: multiplicação, logaritmo, transformações).
2. **Seleção de Features:**  
   Escolher as variáveis mais relevantes para reduzir dimensionalidade.
   - Técnicas comuns: Filtro, Wrapper, Embedded (ex.: LASSO).
3. **Transformação de Features:**  
   Padronizar ou normalizar variáveis numéricas para escalas uniformes.

**Impacto:**  
- Melhora a performance e interpretabilidade dos modelos.
- Reduz o risco de overfitting ao evitar features redundantes.


#### **Seleção de Modelos**

**Descrição:**  
Seleção de modelos é o processo de escolher o modelo que melhor equilibra performance e interpretabilidade.

**Critérios de Avaliação:**
- **$R^2$ Ajustado:** Para modelos de regressão.
- **AUC (Área sob a Curva ROC):** Para modelos de classificação.
- **RMSE (Root Mean Squared Error):** Para medir erros em predições contínuas.
- **Log Loss:** Para avaliar probabilidades preditivas em classificação.

**Trade-offs na Escolha:**
- **Complexidade:** Modelos mais complexos geralmente têm maior capacidade preditiva, mas podem ser difíceis de interpretar.
- **Interpretabilidade:** Modelos mais simples são mais fáceis de explicar, mas podem ter menor capacidade de capturar padrões complexos.



## O Uso de ML em Economia

Os problemas em economia frequentemente envolvem:

* **Previsão de Séries Temporais:** Previsão de variáveis macroeconômicas, como inflação, PIB e taxas de desemprego.
* **Análise de Impacto Causal:** Estimar o efeito de políticas públicas, como subsídios, impostos ou reformas.
* **Classificação e Segmentação:** Identificar padrões em dados de consumidores, mercados ou empresas.
* **Otimização de Decisões:** Melhorar políticas de alocação de recursos ou investimentos.

Técnicas Mais Comuns

* **Previsão:**
  * Modelos de séries temporais: ARIMA, Prophet, Redes Neurais Recorrentes (RNN) e LSTMs.
    * Algoritmos de regressão para prever preços, consumo, ou variáveis socioeconômicas.

* **Causalidade e Inferência:**
  * Double Machine Learning (DML), Modelos de Variáveis Instrumentais, Modelos Baseados em Controle Sintético.
    * Exemplo: Avaliar o impacto de uma política fiscal sobre o PIB.


## Considerações Finais

Machine Learning é uma ferramenta poderosa para analisar dados complexos e extrair insights valiosos em economia. Com a crescente disponibilidade de dados e o desenvolvimento de algoritmos sofisticados, o uso de ML na economia está se tornando cada vez mais comum.

O aprendizado supervisionado é predominante em economia. Isso ocorre por diversos fatores.
* Natureza dos Dados em Economia: Muitos estudos econômicos trabalham com dados rotulados, o que é ideal para aprendizado supervisionado.
* Foco em Predição e Causalidade: Economistas frequentemente estão interessados em prever uma variável dependente ou entender o impacto causal de uma política ou evento. Modelos supervisionados, como regressões, árvores de decisão, e redes neurais, são ferramentas diretas para esses objetivos.
* Interpretação e Explicabilidade: Modelos como regressões lineares, árvores de decisão e random forests são supervisionados e mais interpretáveis, uma característica valorizada em economia.

No entanto, técnicas de aprendizado não supervisionado, semi-supervisionado e por reforço também têm aplicações importantes em economia. Por exemplo, agrupamento de consumidores, redução de dimensionalidade em dados macroeconômicos e otimização de políticas públicas.